<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/PrediccionesModeloGenerado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicciones de 1 día

In [ ]:
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta
import holidays
import pandas as pd
import calendar
# Función para hacer predicciones basada en nuevos datos
def make_prediction(model, scaler_X, scaler_y, new_data):
    if new_data.shape != (9, 17):
        raise ValueError("Los datos de entrada deben tener 9 pasos de tiempo con 17 características cada uno.")
    new_data_scaled = scaler_X.transform(new_data)
    new_data_scaled = new_data_scaled.reshape((1, 9, 17))
    prediction_scaled = model.predict(new_data_scaled)
    prediction = scaler_y.inverse_transform(prediction_scaled)
    return prediction

# Función principal para predecir los valores para un día completo en incrementos de 15 minutos
def predict_for_full_day_dataframe(year, mes, dia, hora_inicio, stations):
    model = load_model('modelo_entrenado.h5')
    scaler_X = joblib.load('scaler_X.pkl')
    scaler_y = joblib.load('scaler_y.pkl')

    fecha_inicio = datetime(year, mes, dia, hora_inicio)
    co_holidays = holidays.CountryHoliday('CO', years=year)
    predictions_list = []

    incrementos = ((24 - hora_inicio) * 4) - 1
    for station_index, station in enumerate(stations):
        for incremento in range(incrementos):
            fecha_actual = fecha_inicio + timedelta(minutes=incremento * 15)
            dia_semana = fecha_actual.weekday() + 1
            es_festivo = 1 if fecha_actual in co_holidays else 0

            new_data_example = np.zeros((9, 17))
            for i in range(9):
                tiempo_incremento = fecha_actual + timedelta(minutes=i * 15)
                new_data_example[i, :6] = [tiempo_incremento.month, tiempo_incremento.day, dia_semana, es_festivo, tiempo_incremento.hour, tiempo_incremento.minute]
                new_data_example[i, 6] = np.random.randint(0, 46)  # Ejemplo de datos aleatorios
                new_data_example[i, 7 + station_index] = 1  # Marcar la estación seleccionada

            prediction = make_prediction(model, scaler_X, scaler_y, new_data_example)
            predictions_list.append({
                "Month": fecha_actual.month,
                "Day": fecha_actual.day,
                "Day_Week_Number": dia_semana,
                "Holidays": es_festivo,
                "Hour": fecha_actual.strftime('%H:%M'),
                "Station": station,
                "Input_Predict": np.ceil(np.abs(prediction.flatten()[0]))
            })
            # Imprimir la fecha, hora y la predicción
            print(f"Fecha y hora: {fecha_actual.strftime('%Y-%m-%d %H:%M')}, Predicción: {prediction.flatten()[0]}")
    predictions_df = pd.DataFrame(predictions_list)
    return predictions_df


# Solicitar al usuario que ingrese los detalles
year = 2024  # Año fijo para este ejemplo
mes = int(input("Ingresa el mes (como número): "))
dia = int(input("Ingresa la fecha del día: "))
hora_inicio = 2

stations = ["Station_(02000)Cabecera Autopista Norte", "Station_(02300)Calle 100 - Marketmedios",
            "Station_(03000)Portal Suba", "Station_(04000)Cabecera Calle 80",
            "Station_(05000)Portal Américas", "Station_(06000)Portal Eldorado",
            "Station_(07000)Portal Sur", "Station_(08000)Portal Tunal",
            "Station_(09000)Cabecera Usme", "Station_(10000)Portal 20 de Julio"]

# Suponiendo que las variables 'year', 'mes', 'dia', 'hora_inicio', y 'stations' están definidas por el usuario, como en tu ejemplo.
predictions_df = predict_for_full_day_dataframe(year, mes, dia, hora_inicio, stations)
# Guardar el DataFrame en un archivo CSV
predictions_df

# Obtener nombre del mes a partir de su número
def get_month_name(month):
    return calendar.month_name[month]

# Obtener nombre del día de la semana a partir de su número (1: lunes, 2: martes, etc.)
def get_day_name(day):
    return calendar.day_name[day-1]
# Convertir los valores numéricos de la columna "Month" a nombres de meses
predictions_df['Month'] = predictions_df['Month'].apply(get_month_name)
# Convertir los valores numéricos de la columna "Day_Week_Number" a nombres de días de la semana
predictions_df['Day_Week_Number'] = predictions_df['Day_Week_Number'].apply(get_day_name)
# Mapear los valores numéricos de la columna "Holidays" a "si" y "no"
predictions_df['Holidays'] = predictions_df['Holidays'].map({1: 'Yes', 0: 'No'})
predictions_df

# Predicciones dentro de un mes


In [ ]:
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta
import holidays
import pandas as pd
import calendar
import time
import psutil

start_time = time.time()
# Función para hacer predicciones basada en nuevos datos
def make_prediction(model, scaler_X, scaler_y, new_data):
    if new_data.shape != (9, 17):
        raise ValueError("Los datos de entrada deben tener 9 pasos de tiempo con 17 características cada uno.")
    new_data_scaled = scaler_X.transform(new_data)
    new_data_scaled = new_data_scaled.reshape((1, 9, 17))
    prediction_scaled = model.predict(new_data_scaled)
    prediction = scaler_y.inverse_transform(prediction_scaled)
    return prediction

# Función principal para predecir los valores para una semana completa en incrementos de 15 minutos
def predict_for_full_week_dataframe(year, mes, dia_inicio, dia_fin, hora_inicio, stations):
    model = load_model('modelo_entrenado.h5')
    scaler_X = joblib.load('scaler_X.pkl')
    scaler_y = joblib.load('scaler_y.pkl')

    fecha_inicio = datetime(year, mes, dia_inicio, hora_inicio)
    fecha_fin = datetime(year, mes, dia_fin, 23, 45)  # Último incremento de 15 minutos del día

    co_holidays = holidays.CountryHoliday('CO', years=year)
    predictions_list = []

    incrementos = int(((fecha_fin - fecha_inicio).total_seconds() / 60) / 15)  # Total de incrementos en la semana
    for station_index, station in enumerate(stations):
        for incremento in range(incrementos):
            fecha_actual = fecha_inicio + timedelta(minutes=incremento * 15)
            dia_semana = fecha_actual.weekday() + 1
            es_festivo = 1 if fecha_actual in co_holidays else 0

            new_data_example = np.zeros((9, 17))
            for i in range(9):
                tiempo_incremento = fecha_actual + timedelta(minutes=i * 15)
                new_data_example[i, :6] = [tiempo_incremento.month, tiempo_incremento.day, dia_semana, es_festivo, tiempo_incremento.hour, tiempo_incremento.minute]
                new_data_example[i, 6] = np.random.randint(0, 46)  # Ejemplo de datos aleatorios
                new_data_example[i, 7 + station_index] = 1  # Marcar la estación seleccionada

            prediction = make_prediction(model, scaler_X, scaler_y, new_data_example)
            predictions_list.append({
                "Month": fecha_actual.month,
                "Day": fecha_actual.day,
                "Day_Week": dia_semana,
                "Holidays": es_festivo,
                "Hour": fecha_actual.strftime('%H:%M'),
                "Station": station,
                "Input_Predict": np.ceil(np.abs(prediction.flatten()[0]))
            })
            # Imprimir la fecha, hora y la predicción
            print(f"Fecha y hora: {fecha_actual.strftime('%Y-%m-%d %H:%M')}, Predicción: {prediction.flatten()[0]}")
    predictions_df = pd.DataFrame(predictions_list)
    return predictions_df

# Obtener nombre del mes a partir de su número
def get_month_name(month):
    return calendar.month_name[month]

# Obtener nombre del día de la semana a partir de su número (1: lunes, 2: martes, etc.)
def get_day_name(day):
    return calendar.day_name[day-1]

# Convertir los valores numéricos de la columna "Month" a nombres de meses
def convert_month_names(predictions_df):
    predictions_df['Month'] = predictions_df['Month'].apply(get_month_name)
    return predictions_df

# Convertir los valores numéricos de la columna "Day_Week_Number" a nombres de días de la semana
def convert_day_names(predictions_df):
    predictions_df['Day_Week'] = predictions_df['Day_Week'].apply(get_day_name)
    return predictions_df

# Cambiar los valores numéricos de la columna "Holidays" a "Yes" y "No"
def convert_holidays(predictions_df):
    predictions_df['Holidays'] = predictions_df['Holidays'].map({1: 'Yes', 0: 'No'})
    return predictions_df

# Solicitar al usuario que ingrese los detalles
year = 2024  # Año fijo para este ejemplo
mes = int(input("Ingresa el mes (como número): "))
dia_inicio = int(input("Ingo el día de inicio de la semana: "))
dia_fin = int(input("Ingresa el día de fin de la semana: "))
hora_inicio = 2
stations = ["Station_(02000)Cabecera Autopista Norte", "Station_(02300)Calle 100 - Marketmedios",
            "Station_(03000)Portal Suba", "Station_(04000)Cabecera Calle 80",
            "Station_(05000)Portal Américas", "Station_(06000)Portal Eldorado",
            "Station_(07000)Portal Sur", "Station_(08000)Portal Tunal",
            "Station_(09000)Cabecera Usme", "Station_(10000)Portal 20 de Julio"]
# Realizar las predicciones para la semana completa
predictions_df = predict_for_full_week_dataframe(year, mes, dia_inicio, dia_fin, hora_inicio, stations)

# Convertir los valores numéricos a nombres de meses, días de la semana y etiquetas más descriptivas
predictions_df = convert_month_names(predictions_df)
predictions_df = convert_day_names(predictions_df)
predictions_df = convert_holidays(predictions_df)

process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
execution_time = end_time - start_time

# Calcular minutos y segundos
minutes = int(execution_time // 60)
seconds = int(execution_time % 60)

# Formatear y mostrar el tiempo de ejecución
print(f"Tiempo de ejecución: {minutes:02d}:{seconds:02d}")
predictions_df